# Clean up
This notebook provides steps and instructions on cleaning up the AWS resources provisioned in this workshop. 

* SageMaker Endpoints for DJL and Embedding model
* Amazon Opensearch serverless collection


> *This notebook has been tested with the **`Data Science 3.0`** kernel, **`Python 3`** with **`ml.t3.medium`** in SageMaker Studio*

# Clean up SageMaker Endpoints

In [ ]:
!pip install sagemaker boto3 -Uq

In [ ]:
import boto3

In [ ]:
%store -r

In [ ]:
sm_client = boto3.client("sagemaker")

In [ ]:
response = sm_client.describe_endpoint(
    EndpointName=llm_endpoint_name)

In [ ]:
data_capture_config = response['DataCaptureConfig']

In [ ]:
# Remove the monitoring schedule, then we could remove the endpoint
if data_capture_config['EnableCapture']:
    response = sm_client.list_monitoring_schedules(EndpointName=llm_endpoint_name)
    monitoring_schedule_name = response['MonitoringScheduleSummaries'][0]['MonitoringScheduleName']
    response = sm_client.delete_monitoring_schedule(MonitoringScheduleName=monitoring_schedule_name)

In [ ]:
import time
response = sm_client.list_monitoring_schedules(EndpointName=llm_endpoint_name)
while True:
    monitoring_summaries = response['MonitoringScheduleSummaries']
    if len(monitoring_summaries) > 0:
        time.sleep(5)
        continue
    break

In [ ]:
response = sm_client.delete_endpoint(
    EndpointName=llm_endpoint_name
)

In [ ]:
response = sm_client.delete_endpoint(EndpointName=embedding_endpoint_name)

In [ ]:
response

In [ ]:
oss_client = boto3.client('opensearchserverless')

In [ ]:
response = oss_client.delete_access_policy(
    name=access_policy_name,
    type='data'
)

In [ ]:
response = oss_client.delete_security_policy(
    name=encryption_policy_name,
    type='encryption'
)

In [ ]:
response = oss_client.delete_security_policy(
    name=network_policy_name,
    type='network'
)

In [ ]:
response = oss_client.list_collections(
    collectionFilters={
        'name': vector_store_name,
        'status': 'ACTIVE'
    },
)

In [ ]:
collection_summaries = response['collectionSummaries']
if len(collection_summaries) > 0:
    collection_id = collection_summaries[0]['id']
    response = oss_client.delete_collection(
        id=collection_id
    )